# wps_ca

A process that runs the [ca.netcdf.wrapper](https://github.com/pacificclimate/ClimDown/blob/master/R/CA.R) function from ClimDown.

**Constructed Analogue (CA) downscaling algorithm:**
    Starts by spatially aggregating high-resolution gridded observations
    up to the scale of a GCM. Then it proceeds to bias correcting the GCM
    based on those observations. Finally, it conducts the search for temporal
    analogues. This involves taking each timestep in the GCM and searching for
    the top 30 closest timesteps in the gridded observations. For each of the
    30 closest "analogue" timesteps, CA records the integer number of the
    timestep (indices) and a weight for each of the analogues.


In [1]:
from birdy import WPSClient
from pkg_resources import resource_filename
from wps_tools.testing import get_target_url
from tempfile import NamedTemporaryFile
from datetime import date
from urllib.request import urlopen
import requests
import os

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "chickadee":
    os.chdir('../')

In [2]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("chickadee")
print(f"Using chickadee on {url}")

Using chickadee on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/chickadee/wps


In [3]:
chickadee = WPSClient(url)

### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [4]:
# NBVAL_IGNORE_OUTPUT
chickadee.ca?

Signature:
chickadee.ca(
    gcm_file,
    obs_file,
    varname,
    indices=None,
    weights=None,
    num_cores='4',
    end_date=datetime.date(2005, 12, 31),
    loglevel='INFO',
)
Docstring:
Constructed Analogue (CA) downscaling algorithm

Parameters
----------
gcm_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of GCM simulations
obs_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of high-res gridded historical observations
varname : string
    Name of the NetCDF variable to downscale (e.g. 'tasmax')
num_cores : {'1', '2', '3', '4'}positiveInteger
    The number of cores to use for parallel execution
end_date : date
    Defines the end of the calibration period
indices : string
    File name to store indices of analogue times steps (suffix .txt)
weights : string
    File name to store weights of analogues (suffix .txt)
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG

### Run CA process

In [6]:
output = chickadee.ca(
    gcm_file=resource_filename("tests", "/data/tiny_gcm.nc"), 
    obs_file=resource_filename("tests", "/data/tiny_obs.nc"), 
    varname="tasmax", 
    end_date=date(1972, 12, 31),
    num_cores=4,
    indices="indices_file.txt",
    weights="weights_file.txt"
)

indices, weights = output.get()

### Test for expected output

In [7]:
# Check that all values are integers
with urlopen(indices) as f:
    indices_output_data = f.read().decode('utf-8')
    assert "." not in indices_output_data

In [8]:
# Check that all values are floats
with urlopen(weights) as f:
    weights_output_data = f.read().decode('utf-8')
    assert weights_output_data.count(".")==len(weights_output_data.split('\n'))-1